# **SVD Comparisons (new C++20 library vs. numpy.linalg.svd)**

## Team H
* Evan Ram
* Prateek Makhija
* James Douthit
* Garrett Hempy

In [6]:
import numpy as np

class FengwangSVD:
    
    @staticmethod
    def build():
        """
        Builds the Fengwang Matrix library SVD tester using Docker.

        New C++20 features are not available on many machines,
        so please have Docker installed.
        """

        # Use bang command in Jupyter notebook since we don't care about this command's output
        !docker build -t fengwang-matrix-svd .
        
        # Technically we don't run any Python in this method
        pass

    def run(self, A):
        """
        Runs the C++ test program for the fengwang/matrix library.
        Returns its command line output as a list of lines from stdout.

        Not using bang command b/c we want to process the output to get timings.
        Timing from the start of this method to the end of it is pointless since
        it includes overhead of process creation.
        
        Takes in a 2d numpy array A to factorize
        """

        self._write_matrix(A)

        import subprocess
        
        cmd = f'docker run -v {self._input_data_path()}:/input.txt fengwang-matrix-svd'.split()
        proc_out = subprocess.check_output(cmd)
        lines = proc_out.decode('utf-8').split('\n')
        results = {}

        for i, line in enumerate(lines):
            # Most lines end with carriage return '\r' for some reason
            line = line.strip()
            
            if i == 0:
                if line == 'SVD Image Program':
                    # Title of program (sanity check that it starts up)
                    continue
                else:
                    # Sanity check failed, so we probably messed up somewhere.
                    raise Exception('Failed to launch program', lines)
    
            if len(line) == 0:
                # Blank line
                continue
        
            [k, v] = line.split(':=', 1)
            k = k.lower().strip()
            v = v.strip()
            
            # We deserialize resulting output matrices to numpy arrays
            if k.startswith('matrix-'):
                v = FengwangSVD._deserialize(v)
            
            # Output numbers should all be integers, to avoid differences in
            # floating point arithmetic between Python and C++ tests
            elif v.isdigit():
                v = int(v)
                
            results[k] = v
            
        self._results = results
    
    @property
    def results(self):
        """
        A dict containing the results from running the program.
        """

        if not hasattr(self, '_results'):
            raise Exception('Please call run() first')

        return self._results
    
    def _input_data_path(self):
        import os
        return os.path.join(os.getcwd(), 'fengwang-matrix/input.txt')
    
    def _write_matrix(self, A):
        """
        Write matrix A as input data to the program.
        """
        
        A_ser = FengwangSVD._serialize(A)
        
        with open(self._input_data_path(), 'wb') as f:
            f.write(f'{A_ser}\n'.encode())

    @staticmethod
    def _serialize(A):
        """
        Serialize 2d numpy array into flat string we can write to our program.
        
        Format:
        {n rows}:{m cols}:{A[1,1]},{A[1,2]},...,{A[2,1]},...,{A[n][m]}
        """
        
        out = f'{A.shape[0]}:{A.shape[1]}:'
        
        for row in A:
            for elt in row:
                out += str(elt) + ','
        
        # Trim final comma
        return out[:-1]
    
    @staticmethod
    def _deserialize(A_ser):
        """
        Deserializes a 2d numpy array from the program.
        
        Format:
        {n rows}:{m cols}:{A[1,1]},{A[1,2]},...,{A[2,1]},...,{A[n][m]}
        """
        
        [n, m, data] = A_ser.split(':')
        data = [float(elem) for elem in data.split(',')]
        data = np.array(data)
        
        return data.reshape((int(n), int(m)))

# Build executable before we can run it
FengwangSVD.build()

A = np.array([[1,2,3],[4,5,6]])
fw_svd = FengwangSVD()
fw_svd.run(A)

print('Results:', fw_svd.results)

Sending build context to Docker daemon  105.3MB
Step 1/5 : FROM gcc:latest
 ---> 2f9778ee181e
Step 2/5 : COPY ./fengwang-matrix /app
 ---> 18f3ed964ae7
Step 3/5 : WORKDIR /app
 ---> Running in 839d34dc00a1
Removing intermediate container 839d34dc00a1
 ---> 5b6c6bcfee70
Step 4/5 : RUN make
 ---> Running in bcc5330b46db
g++ -std=c++2a -Wall -Wextra -O2 -pthread -o svdimage main.cpp -lstdc++fs
Removing intermediate container bcc5330b46db
 ---> dc52870a90dc
Step 5/5 : CMD ["./svdimage"]
 ---> Running in e5d5ca8a88c1
Removing intermediate container e5d5ca8a88c1
 ---> d2c7d0fd03c0
Successfully built d2c7d0fd03c0
Successfully tagged fengwang-matrix-svd:latest
Results: {'indata': '2:3:1,2,3,4,5,6', 'test-var': 'hello', 'some-num': 1234567890}


The following prompts may be useful, but you don't have to use them.

## Introduction

Describe the objective of your study, citing prior work as appropriate (papers, websites, etc.).  There is no requirement on citation style, but please try to be consistent.

## Methods

## Results and interpretation

## Conclusions and open questions